In [1]:
!pip install cryptofeed redis nest_asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.6/610.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 19.4 MB/s eta 0:00:00
  Created wheel for cryptofeed: filename=cryptofeed-2.4.1-cp311-cp311-linux_x86_64.whl size=1601207 sha256=0fe40cf3e9b3994df33bd1d2ca34ff6181d005bdac59ae52fb6fe29515a4f40c
  Stored in directory: /root/.cache/pip/wheels/0a/30/84/dcc1003b05dd7598f9808f2a6e12faeaf4230781aba5c1a205
Successfully built cryptofeed

In [2]:
!pip install -q "cryptofeed" nest_asyncio redis

import logging, nest_asyncio, asyncio, json, time
from decimal import Decimal
from datetime import datetime
from cryptofeed import FeedHandler
from cryptofeed.defines import TRADES, L2_BOOK, OPEN_INTEREST, BID, ASK
from cryptofeed.exchanges import OKX
import redis

# Silence the very chatty underlying loggers
logging.getLogger('cryptofeed').setLevel(logging.WARNING)
nest_asyncio.apply()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.6/610.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 15.1 MB/s eta 0:00:00


In [ ]:
# ────────────────────────────────────────────────────────────────
#  CryptoTradeLogger – single‑file collector for OKX BTC‑USDT‑PERP
#  Compatible with Google Colab or any regular Python 3.9+ runtime
# ────────────────────────────────────────────────────────────────
# Optional (uncomment in Colab):
# !pip install -q "cryptofeed[redis]" redis nest_asyncio

import asyncio, json, logging, time
from datetime import datetime
from decimal import Decimal

import nest_asyncio
import redis
from cryptofeed import FeedHandler
from cryptofeed.defines import L2_BOOK, OPEN_INTEREST, TRADES
from cryptofeed.exchanges import OKX

nest_asyncio.apply()                    # allow nested loops (Colab/Jupyter)

# ─── Redis connection ───────────────────────────────────────────
redis_client = redis.Redis(
    host="redis-19129.c262.us-east-1-3.ec2.redns.redis-cloud.com",
    port=19129,
    username="satvik",
    password="Stankar@123",
    decode_responses=True,
)

# ─── Reduce Cryptofeed noise ────────────────────────────────────
for name in ("cryptofeed", "feedhandler", "asyncio"):
    logging.getLogger(name).setLevel(logging.WARNING)

# ─── In‑memory OHLCV buffer (one second granularity) ────────────
candles: dict[int, dict] = {}


def sanitize(obj):
    "Recursively cast Decimals to float."
    if isinstance(obj, Decimal):
        return float(obj)
    if isinstance(obj, dict):
        return {k: sanitize(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [sanitize(i) for i in obj]
    return obj


async def publish_candles():
    "Every second flush the previous second's OHLCV to Redis."
    while True:
        now, prev = int(time.time()), int(time.time()) - 1
        if (c := candles.pop(prev, None)) is not None:
            c["start"] = datetime.fromtimestamp(prev).isoformat()
            redis_client.zadd("okx:candles", {json.dumps(c): prev})
            print(f"🕯️  {c}")
        await asyncio.sleep(1)


async def handle_trade(t, receipt_ts):
    exch_ts = datetime.fromtimestamp(t.timestamp).isoformat()
    rec_ts = datetime.fromtimestamp(receipt_ts).isoformat()

    doc = {
        "exchange": t.exchange,
        "symbol": t.symbol,
        "side": t.side,
        "price": float(t.price),
        "amount": float(t.amount),
        "exchange_ts": exch_ts,
        "received_ts": rec_ts,
    }
    redis_client.zadd("okx:trades", {json.dumps(doc): t.timestamp})
    print(f"💱  {doc}")

    # OHLCV aggregation (1‑second buckets)
    sec, price, vol = int(t.timestamp), float(t.price), float(t.amount)
    if sec in candles:
        c = candles[sec]
        c["high"] = max(c["high"], price)
        c["low"] = min(c["low"], price)
        c["close"] = price
        c["volume"] += vol
    else:
        candles[sec] = {
            "open": price,
            "high": price,
            "low": price,
            "close": price,
            "volume": vol,
        }


async def handle_book(book, receipt_ts):
    """
    Serialise the entire L2 book using the built‑in helper.
    This avoids direct interaction with the internal SortedDict structure
    that caused the earlier 'no attribute items' crash. You can pass
    delta=True if you only need updates.
    """
    book_dict = book.to_dict(numeric_type=float)     # ← safe helper :contentReference[oaicite:1]{index=1}
    book_dict["timestamp"] = datetime.fromtimestamp(receipt_ts).isoformat()
    redis_client.zadd("okx:book", {json.dumps(book_dict): receipt_ts})
    print(f"📖  Book snapshot @ {book_dict['timestamp']}")


async def handle_open_interest(oi, receipt_ts):
    doc = {
        "exchange": oi.exchange,
        "symbol": oi.symbol,
        "open_interest": float(oi.open_interest),
        "timestamp": datetime.fromtimestamp(receipt_ts).isoformat(),
    }
    redis_client.zadd("okx:open_interest", {json.dumps(doc): receipt_ts})
    print(f"📊  {doc}")


def main():
    fh = FeedHandler()
    loop = asyncio.get_event_loop()
    loop.create_task(publish_candles())

    fh.add_feed(
        OKX(
            symbols=["BTC-USDT-PERP"],
            channels=[TRADES, L2_BOOK, OPEN_INTEREST],
            callbacks={
                TRADES: handle_trade,
                L2_BOOK: handle_book,
                OPEN_INTEREST: handle_open_interest,
            },
        )
    )
    fh.run()                                     # blocks forever


if __name__ == "__main__":
    main()


📊  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'open_interest': 3187802.1199999996, 'timestamp': '2025-04-22T16:42:13.534763'}
📖  Book snapshot @ 2025-04-22T16:42:13.886506
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90715.4, 'amount': 0.04, 'exchange_ts': '2025-04-22T16:42:13.540000', 'received_ts': '2025-04-22T16:42:14.016569'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90715.5, 'amount': 0.05, 'exchange_ts': '2025-04-22T16:42:13.557000', 'received_ts': '2025-04-22T16:42:14.080227'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90715.4, 'amount': 1.29, 'exchange_ts': '2025-04-22T16:42:13.599000', 'received_ts': '2025-04-22T16:42:14.143623'}
📖  Book snapshot @ 2025-04-22T16:42:14.207327
📖  Book snapshot @ 2025-04-22T16:42:14.273562
🕯️  {'open': 90715.4, 'high': 90715.5, 'low': 90715.4, 'close': 90715.4, 'volume': 1.3800000000000001, 'start': '2025-04-22T16:42:13'}
📖  Book snapshot @ 2025-04-2

2025-04-22 16:44:49,318 : ERROR : OKX.ws.5: encountered an exception, reconnecting in 1.0 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 65, in _create_connection
    await self._handler(connection, self.handler)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 99, in _handler
    await handler(message, connection, self.conn.last_message)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 422, in message_handler
    await self._book(msg, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 306, in _book
    await self.book_callback(L2_BOOK, self._l2_book[pair], timestamp, timestamp=self.timestamp_normalize(int(update['ts'])), raw=msg, delta=delta, checksum=update['checksum'] & 0xFFFFFFFF)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/feed.py", line 247, in book_callback
    await 

📊  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'open_interest': 3189449.9399999995, 'timestamp': '2025-04-22T16:44:51.119216'}
📖  Book snapshot @ 2025-04-22T16:44:51.479045
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90806.8, 'amount': 1.87, 'exchange_ts': '2025-04-22T16:44:51.323000', 'received_ts': '2025-04-22T16:44:51.609025'}
📖  Book snapshot @ 2025-04-22T16:44:51.672862
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90806.8, 'amount': 1.0, 'exchange_ts': '2025-04-22T16:44:51.443000', 'received_ts': '2025-04-22T16:44:51.738467'}
📖  Book snapshot @ 2025-04-22T16:44:51.802147
📖  Book snapshot @ 2025-04-22T16:44:51.868973
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90806.9, 'amount': 0.26, 'exchange_ts': '2025-04-22T16:44:51.657000', 'received_ts': '2025-04-22T16:44:51.934755'}
📖  Book snapshot @ 2025-04-22T16:44:51.998164
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price

2025-04-22 16:44:52,952 : ERROR : OKX.ws.5: encountered an exception, reconnecting in 1.0 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 65, in _create_connection
    await self._handler(connection, self.handler)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 99, in _handler
    await handler(message, connection, self.conn.last_message)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 422, in message_handler
    await self._book(msg, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 306, in _book
    await self.book_callback(L2_BOOK, self._l2_book[pair], timestamp, timestamp=self.timestamp_normalize(int(update['ts'])), raw=msg, delta=delta, checksum=update['checksum'] & 0xFFFFFFFF)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/feed.py", line 247, in book_callback
    await 

🕯️  {'open': 90806.8, 'high': 90806.9, 'low': 90806.8, 'close': 90806.9, 'volume': 0.6000000000000001, 'start': '2025-04-22T16:44:52'}
📊  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'open_interest': 3189530.84, 'timestamp': '2025-04-22T16:44:54.800769'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90816.9, 'amount': 4.77, 'exchange_ts': '2025-04-22T16:44:54.672000', 'received_ts': '2025-04-22T16:44:54.864178'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90816.9, 'amount': 4.12, 'exchange_ts': '2025-04-22T16:44:54.730000', 'received_ts': '2025-04-22T16:44:54.929054'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90816.8, 'amount': 5.83, 'exchange_ts': '2025-04-22T16:44:54.797000', 'received_ts': '2025-04-22T16:44:54.992346'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90816.9, 'amount': 0.07, 'exchange_ts': '2025-04-22T16:44:54.814000', 'received_ts': '2025-04-22T16:4

...

2025-04-22 16:45:57,155 : ERROR : OKX.ws.5: encountered an exception, reconnecting in 1.0 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 65, in _create_connection
    await self._handler(connection, self.handler)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 99, in _handler
    await handler(message, connection, self.conn.last_message)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 422, in message_handler
    await self._book(msg, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 285, in _book
    await self.book_callback(L2_BOOK, self._l2_book[pair], timestamp, timestamp=self.timestamp_normalize(int(update['ts'])), checksum=update['checksum'] & 0xFFFFFFFF, raw=msg)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/feed.py", line 247, in book_callback
    await self.callback

📊  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'open_interest': 3193754.73, 'timestamp': '2025-04-22T16:45:58.882233'}


2025-04-22 16:45:59,341 : ERROR : OKX.ws.5: encountered an exception, reconnecting in 1.0 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 65, in _create_connection
    await self._handler(connection, self.handler)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 99, in _handler
    await handler(message, connection, self.conn.last_message)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 422, in message_handler
    await self._book(msg, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 285, in _book
    await self.book_callback(L2_BOOK, self._l2_book[pair], timestamp, timestamp=self.timestamp_normalize(int(update['ts'])), checksum=update['checksum'] & 0xFFFFFFFF, raw=msg)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/feed.py", line 247, in book_callback
    await self.callback

📊  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'open_interest': 3193707.1599999997, 'timestamp': '2025-04-22T16:46:01.088612'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90784.1, 'amount': 0.94, 'exchange_ts': '2025-04-22T16:46:01.010000', 'received_ts': '2025-04-22T16:46:01.152580'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'buy', 'price': 90784.1, 'amount': 4.8, 'exchange_ts': '2025-04-22T16:46:01.021000', 'received_ts': '2025-04-22T16:46:01.216236'}
💱  {'exchange': 'OKX', 'symbol': 'BTC-USDT-PERP', 'side': 'sell', 'price': 90784.0, 'amount': 2.54, 'exchange_ts': '2025-04-22T16:46:01.028000', 'received_ts': '2025-04-22T16:46:01.279627'}


2025-04-22 16:46:01,581 : ERROR : OKX.ws.5: encountered an exception, reconnecting in 1.0 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 65, in _create_connection
    await self._handler(connection, self.handler)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/connection_handler.py", line 99, in _handler
    await handler(message, connection, self.conn.last_message)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 426, in message_handler
    await self._trade(msg, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/exchanges/okx.py", line 258, in _trade
    await self.callback(TRADES, t, timestamp)
  File "/usr/local/lib/python3.11/dist-packages/cryptofeed/feed.py", line 258, in callback
    await cb(obj, receipt_timestamp)
  File "<ipython-input-7-4a718af108f1>", line 72, in handle_trade
    redis_client.zadd("okx:trades", {json.dumps(doc):